# 執行腳本

In [ ]:
%load_ext autoreload
%autoreload 2

import data_loader
import feature_eng  

# 1. 載入資料 (採購)
df_raw = data_loader.load_and_merge_data()

# 2. 增加特徵 (加工)
df_processed = feature_eng.add_technical_indicators(df_raw)

# 3. 檢查結果
# 我們只看某一支股票 (例如 1101) 的新欄位就好，不然太多了
cols_to_check = [c for c in df_processed.columns if '1101' in c]
print(df_processed[['date'] + cols_to_check].head(20))